In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.data import AUTOTUNE

# Load the trained model
model = load_model(r'/mnt/c/SV-Desktop/FALL SEMESTER 2024-25/Dissertation 1/Implementation/Models/Raw/EfficientNet/EfficientNetB0_RAW_wo_aug_1.6.keras')

# Define the class names (should match the classes in the original dataset)
class_names = ['anthracnose', 'healthy', 'leaf miner', 'red rust']


2024-09-16 20:15:54.454464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-16 20:15:54.463139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 20:15:54.472464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 20:15:54.475041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 20:15:54.482054: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Function to preprocess a single image for real-time predictions
def preprocess_image(image_path, img_size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read the image at {image_path}")
    
    # Convert from BGR (OpenCV) to RGB (used by ImageDataGenerator)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize to 224x224 like in ImageDataGenerator
    image = cv2.resize(image, img_size)
    
    # Normalize the image to match rescale=1./255 in the test generator
    image = image.astype('float16') / 255.0
    
    # Add batch dimension (1, 224, 224, 3)
    image = np.expand_dims(image, axis=0)
    return image


In [3]:
# Define image size expected by the model
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Function to load the test dataset using tf.data.Dataset
def load_test_dataset(test_dir, img_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE):
    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        image_size=img_size,
        batch_size=batch_size,
        label_mode='categorical',  # Matches the categorical setup during training
        shuffle=False
    )
    
    # Normalize the images (rescale=1./255)
    test_dataset = test_dataset.map(lambda x, y: (x / 255.0, y), num_parallel_calls=AUTOTUNE)
    
    # Cache and prefetch for efficiency
    test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)
    
    return test_dataset

# Load the test dataset from the directory
test_dir = r'/mnt/c/SV-Desktop/FALL SEMESTER 2024-25/Dissertation 1/Implementation/Dataset/Dataset for Crop Pest and Disease Detection/Raw Data/CCMT Dataset/Cashew/test'
test_dataset = load_test_dataset(test_dir)


Found 927 files belonging to 4 classes.


In [4]:
# Path to the real-time test image
image_path = r'/mnt/c/SV-Desktop/FALL SEMESTER 2024-25/Dissertation 1/Implementation/testing/healthy_1.jpeg'

# Ensure the file exists
if not os.path.exists(image_path):
    raise FileNotFoundError(f"The image path {image_path} does not exist.")

# Predict and display the result for real-time image
preprocessed_image = preprocess_image(image_path)
prediction = model.predict(preprocessed_image)
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class = class_names[predicted_class_index]

print(f'The predicted class for the real-time image is: {predicted_class}')
print(f'Prediction probabilities for real-time image: {prediction}')


I0000 00:00:1726497973.111258    6051 service.cc:146] XLA service 0x7efc80046a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726497973.111289    6051 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-09-16 20:16:13.167692: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-16 20:16:13.479977: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
The predicted class for the real-time image is: anthracnose
Prediction probabilities for real-time image: [[nan nan nan nan]]


I0000 00:00:1726497978.150629    6051 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [5]:
# Evaluate the model on the entire test dataset
final_results = model.evaluate(test_dataset)
print(f"Final Test Loss: {final_results[0]}")
print(f"Final Test Accuracy: {final_results[1]}")


29/29 ━━━━━━━━━━━━━━━━━━━━ 15s 241ms/step - accuracy: 0.0537 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00
Final Test Loss: nan
Final Test Accuracy: 0.27292340993881226


In [6]:
# Iterate through the test dataset and predict
for test_images, true_labels in test_dataset.take(1):  # Take one batch from the dataset
    test_predictions = model.predict(test_images)
    
    # Get predicted class indices
    test_predicted_class_indices = np.argmax(test_predictions, axis=1)
    
    # Map class indices to class names
    test_predicted_classes = [class_names[idx] for idx in test_predicted_class_indices]
    
    print(f'Test generator predicted classes: {test_predicted_classes}')
    print(f'Test generator prediction probabilities for the first image: {test_predictions[0]}')
    break  # Only checking the first batch for this example


2024-09-16 20:16:45.105377: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2281', 164 bytes spill stores, 164 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Test generator predicted classes: ['anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose', 'anthracnose']
Test generator prediction probabilities for the first image: [nan nan nan nan]
